In [1]:
from patching import (
    import_shapefile_for_patches,
    do_the_patching,
    store_coco_ground_truth,
    import_image,
)

/home/robert/.pyenv/versions/ds-wildfire/lib/python3.8/site-packages/typeguard/__init__.py:834: UserWarning: no type annotations present -- not typechecking patching.import_shapefile_for_patches
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [2]:
from patchutils.coco import save_gt_overlaid
from docopt import docopt
import os
import glob
import rasterio
import geopandas as gp
from matplotlib import pyplot
from datetime import datetime
import geopandas as gpd

num_patches=1000
patch_size=128
scenes_path="/home/robert/ds-wildfire/data/scenes"
scenes="31/T/GJ"
scenes_string="_31_T_GJ_"
shapefile="/data/raw_data/wildfires-ground-truth/france/vars/N_DFCI_CONTOUR_FEUX_2017_S_083.shp"
save_path="/home/robert/ds-wildfire/data"
class_name="DATE_ECLOS"

In [3]:
date = {}
date_datetime={}
date_string={}

In [4]:
    path_to_store_patches = os.path.join(save_path, "patches")
    os.makedirs(path_to_store_patches, exist_ok=True)
    path_to_store_anno = os.path.join(save_path, "anno")
    os.makedirs(path_to_store_anno, exist_ok=True)
    path_to_store_burn_vis = os.path.join(save_path, "burn_vis")
    os.makedirs(path_to_store_burn_vis, exist_ok=True)

In [5]:
path_to_scene = os.path.join(
            scenes_path, scenes)
        
list_of_paths = glob.glob(
            path_to_scene + "/*/*/*/" + "*.jp2".format(scenes)
        )

In [6]:
path_to_scene

'/home/robert/ds-wildfire/data/scenes/31/T/GJ'

In [7]:
list_of_paths

['/home/robert/ds-wildfire/data/scenes/31/T/GJ/2015/8/6/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2015/8/12/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2015/7/30/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2016/7/11/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2016/7/21/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2016/7/17/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2016/7/14/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2017/6/16/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2017/6/19/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2017/6/12/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2017/9/7/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2017/9/4/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2017/9/5/B8AB11B12.jp2',
 '/home/robert/ds-wildfire/data/scenes/31/T/GJ/2017/7/2

In [13]:
for date_var in range (0, len(list_of_paths)):
    date_index = list_of_paths[date_var].index("201")
    date_end_index = list_of_paths[date_var].index("/B8A")
    date[date_var] = list_of_paths[date_var][date_index:date_end_index]
    date_datetime[date_var] = datetime.strptime(date[date_var], '%Y/%m/%d')
    date_string[date_var]=date_datetime[date_var].strftime('%Y-%m-%d')
    raster = import_image(list_of_paths[date_var])
    raster_meta = raster.meta
    
    gdf = gpd.read_file(shapefile)
    shapefile_date=gdf.loc[(gdf['DATE_ECLOS']<date_string[date_var])]
    
    patch_dfs, patch_windows = import_shapefile_for_patches(
        shapefile_date, raster, raster_meta, patch_size, num_patches, scenes_string + date_string[date_var]
    )
    
    do_the_patching(
                raster,
                path_to_store_patches,
                patch_windows.keys(),
                patch_windows.values(),
                bands=[3,2,1],
    )
    
    store_coco_ground_truth(
                path_to_store_anno, patch_dfs, patch_size, class_name, scenes_string + date_string[date_var]
    )
    try:
        save_gt_overlaid(
            os.path.join(path_to_store_anno, "anno{}.json".format(scenes_string + date_string[date_var])),
            path_to_store_patches,
            path_to_store_burn_vis,
            )
    except Exception as e:
                print(e)
                print("MGRS tile without annotations: {}".format(scenes_string + date_string[date_var]))
    raster.close()

1764it [00:04, 424.66it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

Writing new json file... /home/robert/ds-wildfire/data/anno/anno_31_T_GJ_2015-08-06.json
'annotations'
MGRS tile without annotations: _31_T_GJ_2015-08-06


1764it [00:04, 424.21it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

Writing new json file... /home/robert/ds-wildfire/data/anno/anno_31_T_GJ_2015-08-12.json
'annotations'
MGRS tile without annotations: _31_T_GJ_2015-08-12


1764it [00:04, 425.90it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

...

Writing new json file... /home/robert/ds-wildfire/data/anno/anno_31_T_GJ_2017-10-09.json


/home/robert/ds-wildfire/preparedata/patchutils/geo.py:146: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(geojson["coordinates"]), precision
1764it [00:04, 407.38it/s]
13it [00:00, 220.27it/s]


Writing new json file... /home/robert/ds-wildfire/data/anno/anno_31_T_GJ_2017-10-07.json


/home/robert/ds-wildfire/preparedata/patchutils/geo.py:146: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(geojson["coordinates"]), precision
1764it [00:04, 397.20it/s]
15it [00:00, 296.65it/s]


Writing new json file... /home/robert/ds-wildfire/data/anno/anno_31_T_GJ_2017-10-10.json


In [14]:
scenes

'31/T/GJ'

In [35]:
scene_list=list(scenes)
scene_list[2]="_"
scene_list[4]="_"
scene_list.insert(0,"_")
scene_list.insert(8, "_")
scene_string=""
scene_string=scene_string.join(scene_list)

In [36]:
scene_string

'_31_T_GJ_'

In [24]:
"".join(scene_string)

'31_T_GJ'